# RTE (Recognizing Textual Entailment) with DeBERTa
## Using a pretrained DeBERTa model fine-tuned on MNLI for zero-shot text classification on SNLI
Inspired by Keras code example [Semantic Similarity with BERT](https://keras.io/examples/nlp/semantic_similarity_with_bert/)

Executed on AWS SageMaker `ml.g4dn.2xlarge` GPU instance

## Setup

In [1]:
# !pip install torch setfit wandb 

In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss
from datasets import load_dataset

/Users/thierry.wendling/Github/ml-examples/natural-language/nli/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom dataset

In [3]:
# dataset = load_dataset("SetFit/SentEval-CR")
dataset = load_dataset("emotion")
dataset['train'][0]

Using custom data configuration default
Reusing dataset emotion (/Users/thierry.wendling/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)
100%|██████████| 3/3 [00:00<00:00, 541.32it/s]


{'text': 'i didnt feel humiliated', 'label': 0}

In [4]:
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights.


In [6]:
# Simulate the few-shot regime by sampling 8 examples per class
num_classes = 6
train_ds = dataset["train"].shuffle(seed=42).select(range(8 * num_classes))
test_ds = dataset["test"]

# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    metric='accuracy',
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20,
    num_epochs=1, # The number of text pairs to generate
    learning_rate=2e-5
)

# Train and evaluate
trainer.train()
metrics = trainer.evaluate()
metrics

Loading cached shuffled indices for dataset at /Users/thierry.wendling/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-db0d0011cbbf5938.arrow


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights.
48 train samples in total, 120 train steps with batch size 16


Epoch: 100%|██████████| 1/1 [07:08<00:00, 428.07s/it]


{'accuracy': 0.5265}